In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__, "..")); Pkg.instantiate();
using Altro
using TrajectoryOptimization
using RobotDynamics
using StaticArrays
using LinearAlgebra
using Rotations
const TO = TrajectoryOptimization

  Activating environment at `~/.julia/dev/AltroTutorials/Project.toml`
┌ Info: Precompiling Altro [5dcf52e5-e2fb-48e0-b826-96f46d2e3e73]
└ @ Base loading.jl:1317


TrajectoryOptimization

# Rocket MPC
In this problem we'll walk through how to set up a simple rocket model, including second-order cone constraints, generate a reference trajectory, and then track that reference trajectory with Conic MPC to solve the soft-landing problem. We will also cover how to define your own constraints.


## 1. Defining the Model
For this problem we'll use a simple linear model of a rocket flying vertically. This simplified model is a decent approximation when the roll angle is small.
This exposes part of the API for linear models defined by RobotDynamics.jl. The call to `LinearizedModel` can be used with any model, including nonlinear ones, to generate a discrete linear or linear-affine model.

In [2]:
function RocketModel(mass, grav, dt, ωPlanet = [0.0; 0.0; 0.0];
        integration=RobotDynamics.Exponential)
    
    # Define the dynamics matrices (including the affine term)
    A = [
        zeros(3,3)      I(3);
        -Rotations.skew(ωPlanet)^2   -2*Rotations.skew(ωPlanet);
    ]
    B = [
        zeros(3,3);
        1/mass * I(3)
    ]
    d = [
        zeros(3);
        grav
    ]

    # Continuous Linear-Affine Model
    cmodel =  LinearModel(A,B,d)

    # Discrete Model
    model = LinearizedModel(cmodel, dt=dt, is_affine=true, integration=integration)
end

# Generate the model
mass = 10.0             # kg
ωPlanet = [0, 0, 0]     # rad/s
gravity = [0, 0, -9.81]  # m/s/s
N = 301                 # number of knot points
dt = 0.05               # sec
tf = (N-1)*dt           # sec
model = RocketModel(mass, gravity, dt, ωPlanet);
n,m = size(model)

# TIP: Get the A,B matrices for the discrete model
RobotDynamics.get_A(model)
RobotDynamics.get_B(model);

## 2. Define the Objective

In [3]:
Q = Diagonal(@SVector fill(1e-2, n))
R = Diagonal(@SVector fill(1e-0, m))
Qf = Diagonal(@SVector fill(1e4, n))
xf = @SVector zeros(n)
x0 = SA_F64[4.0, 2.0, 20.0, -3.0, 2.0, -5.0]  # km
obj = LQRObjective(Q, R, Qf, xf, N);

## 3. Define the Constraints
For the constraints, we'll start off with the ones that we can define using the constraints [already defined in TrajectoryOptimization.jl](http://roboticexplorationlab.org/TrajectoryOptimization.jl/dev/constraint_api.html#Implemented-Constraints).

In [4]:
cons = ConstraintList(n, m, N)

# Goal Constraint
TO.add_constraint!(cons, GoalConstraint(xf), N)

# Maximum Thrust
# ||u||₂ <= u_bnd
perWeightMax = 2.0  
u_bnd = mass * abs(gravity[3]) * perWeightMax
maxThrust = NormConstraint(n, m, u_bnd, TO.SecondOrderCone(), :control)
TO.add_constraint!(cons, maxThrust, 1:N-1)

### Defining a Custom Constraint
For the next constraints, we need a custom constraint that can handle a more general SOCP constraint. We'll define a constraint of the form:
$$ || A z ||_2 \leq c^T z $$
where $z$ is the concatenated vector of the states and controls.

For a second-order cone constaint of the form $||v||_2 \leq c$, we define the output of the constraint to be $\begin{bmatrix} v^T & c \end{bmatrix}^T$. The Jacobian of the constraint is then just the Jacobian of this vector with respect to the inputs.

When defining our constraint below, we specify that it is a subclass of `StageConstraint`, meaning it is a function of the states and controls of a single knot point. Most other constraints will either be `StateConstraint` or `ControlConstraint`. This just tells the solver how wide the non-zero block in the constraint Jacobian is. Additionally, we need to provide methods for querying the state, control, and constraint output dimensions. Then, obviously, we need to define the constraint function via the `evaluate` method, which can be of any of the following forms:
* `evaluate(con, z::KnotPoint)` - Stage, state, and control constraints
* `evaluate(con, x::StaticVector, u::StaticVector)` - Stage constraints
* `evaluate(con, x::StaticVector)` - State constraints
* `evaluate(con, u::ControlVector)` - Control constraints

Since it's convenient to index out of the concatentated state and control vector stored in the `KnotPoint` type, we use the first method. 

We also need to define the Jacobian via the `jacobian!` method. Jacobians are only automatically defined using [ForwardDiff.jl](https://github.com/JuliaDiff/ForwardDiff.jlhttps://github.com/JuliaDiff/ForwardDiff.jl) for state and control constraints.

In [5]:
# Is a sub-class of StageConstraint since it is a function of both the states and controls
struct LinearNormConstraint{N,D,ND} <: TO.StageConstraint
    n::Int  # state dimension
    m::Int  # control dimension
    A::SMatrix{D, N, Float64, ND}
    c::SVector{N, Float64}
    inds::SVector{N, Int}  # extracts out a portion of the z vector
    function LinearNormConstraint{N,D}(n::Int, m::Int, A::AbstractMatrix, c::AbstractVector, inds=SVector{n+m}(1:n+m)) where {N,D}
        if inds == :state
            inds = SVector{n}(1:n)
        elseif inds == :control
            inds = SVector{m}(n .+ (1:m))
        end
        new{N,D,N*D}(n, m, SMatrix{D,N}(A), SVector{N}(c), inds)
    end
end
TO.state_dim(con::LinearNormConstraint) = con.n
TO.control_dim(con::LinearNormConstraint) = con.m
TO.sense(::LinearNormConstraint) = TO.SecondOrderCone()
TO.Base.length(::LinearNormConstraint{<:Any,D}) where D = D + 1

function TO.evaluate(con::LinearNormConstraint, z::KnotPoint)
    x = z.z[con.inds]
    v = con.A*x
    s = con.c'x
    return push(v, s)  # stack the vector and scalar parts
end

function TO.jacobian!(∇c, con::LinearNormConstraint{N,D}, z::KnotPoint) where {N,D}
    ∇c[1:D,con.inds] .= con.A
    ∇c[D+1,con.inds] .= con.c
    return true  # must return a boolean. `true` if the Jacobian is constant, `false` otherwise
end

In [6]:
# Max Angle angle
# || [u_x, u_y] || ≤ tan(θ) u_z
θ_thrust_max = 5.0 # deg
α_max = tand(θ_thrust_max)

AThrust = SA_F64[
    1 0 0;
    0 1 0;
    0 0 0
]
cThrust = SA_F64[0, 0, α_max]
maxAngle = LinearNormConstraint{3,3}(n, m, AThrust, cThrust, :control)
TO.add_constraint!(cons, maxAngle, 1:N-1)

# Glideslope Constraint
# Keeps the rocket within a conic airspace, with the tip at the landing site
# Keeps the rocket from flying low to the ground if it's not near the landing site
θ_glideslope = 45 # deg
α_glide = tand(θ_glideslope)
AGlide = SA_F64[
    1 0 0;
    0 1 0;
    0 0 0;
]
cGlide = SA_F64[0, 0, α_glide]
glideSlope = LinearNormConstraint{3,3}(n, m, AGlide, cGlide, SA[1,2,3])
TO.add_constraint!(cons, glideSlope, 1:N-1)

## 3. Define the problem
We can now define our problem. Since our dynamics are already discretized, we want to disable the automatic integration by setting the integration method to `RobotDynamics.PassThrough`.

In [7]:
prob = Problem(model, obj, xf, tf, x0=x0, constraints=cons, integration=RobotDynamics.PassThrough);
U0 = [-mass * gravity for k = 1:N-1]
initial_controls!(prob, U0)
rollout!(prob)

## 4. Solve for the Reference Trajectory
We now use Altro.jl to solve for our 15-second reference trajectory. Generally, this step is done offline and uses nonlinear dynamics to get a high-quality reference for use when tracking it with MPC.

In [11]:
opts = SolverOptions(
    cost_tolerance_intermediate=1e-4,
    penalty_scaling=500.,
    penalty_initial=1e-2,
#     verbose = 1,
    projected_newton = false,
    constraint_tolerance = 1.0e-5,
    iterations = 5000,
    iterations_inner = 100,
    iterations_linesearch = 100,
    iterations_outer = 500,
)
solver = ALTROSolver(prob, opts);


In [12]:
initial_controls!(solver, U0)
solve!(solver);


SOLVE COMPLETED
 solved using the ALTRO Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 24
    Solve Time: 22.920275 (ms)

  Covergence
    Terminal Cost: 77424.12653663858
    Terminal dJ: 2.1827872842550278e-10
    Terminal gradient: 0.0016115446952415723
    Terminal constraint violation: 1.3759199739560302e-6
    Solve Status: SOLVE_SUCCEEDED


In [13]:
Z_reference = get_trajectory(solver);

## 5. Generate the MPC Problem
We now generate our MPC problem. This is nearly the same as the problem set up before, but we are now penalizing deviations from the reference trajectory, and remove the goal constraint. The function provided below performs this step. Note the use of `TO.TrackingObjective`, which conveniently creates an objective to track the reference trajectory.

In [16]:
"""
Create a Trajectory Optimization problem that tracks the trajectory in `prob`,
using the same constraints, minus the goal constraint. Tracks the first `N`
time steps.
"""
function gen_tracking_problem(prob::TO.Problem, N;
        Qk = 10.0,
        Rk = 0.1,
        Qfk = Qk,
    )
    n,m = size(prob)
    dt = prob.Z[1].dt
    tf = (N-1)*dt

    # Get sub-trajectory
    Z = Traj(prob.Z[1:N])
    x0 = state(Z[1])
    xf = state(Z[N])  # this actually doesn't effect anything

    # Generate a cost that tracks the trajectory
    Q = Diagonal(@SVector fill(Qk, n))
    R = Diagonal(@SVector fill(Rk, m))
    Qf = Diagonal(@SVector fill(Qfk, n))
    obj = TO.TrackingObjective(Q, R, Z, Qf=Qf)

    # Use the same constraints, except the Goal constraint
    cons = ConstraintList(n,m,N)
    for (inds, con) in zip(prob.constraints)
        if !(con isa GoalConstraint)
            if inds.stop > N
                inds = inds.start:N-(prob.N - inds.stop)
            end
            length(inds) > 0 && TO.add_constraint!(cons, con, inds)
        end
    end

    prob = TO.Problem(prob.model, obj, xf, tf, x0=x0, constraints=cons,
        integration=TO.integration(prob)
    )
    initial_trajectory!(prob, Z)
    return prob
end

Now we're ready to run our MPC problem. Our approach here is simplistic, but gets across the key ideas. First, we set up our solver and solve the first iteration. Then, online, we take the first control from the previous solve, integrate our dynamics forward with some noise to simulate the movement of our "real" system (generally you'd want to use a high-fidelity model for this step), and then update our MPC optimization problem by shifting the portion of the reference trajectory we're tracking and updating our initial state (which generally will be provided by a state estimator running on the real system). For slightly better performance, we also shift our initial guesses for both the state/control trajectory and the dual variables by one time step. The problem is solved and the process is repeated until the last time step of our MPC problem reaches the end of the reference trajectory. In general you would run this to obtain some steady-state behavior past the end of the reference trajectory.

In [20]:
function run_MPC(prob_mpc, opts_mpc, Z_track,
                 num_iters = length(Z_track) - prob_mpc.N)
    # Instantiate the solver
    altro = ALTROSolver(prob_mpc, opts_mpc)

    # Solve initial iteration
    Altro.solve!(altro)

    # Pre-allocate some storage
    iters = zeros(Int, num_iters)
    times = zeros(num_iters)

    # Get the problem state size and control size
    n,m = size(prob_mpc)

    # Some setup prior to the MPC loop
    t0 = 0
    k_mpc = 1
    x0 = SVector(prob_mpc.x0)
    X_traj = [copy(x0) for k = 1:num_iters+1]

    # Begin the MPC LOOP
    for i = 1:num_iters
        # Update initial time
        t0 += dt
        k_mpc += 1
        TO.set_initial_time!(prob_mpc, t0)

        # Update initial state by using 1st control, and adding some noise
        x0 = discrete_dynamics(TO.integration(prob_mpc),
                                    prob_mpc.model, prob_mpc.Z[1])

        noise_pos = @SVector randn(3)
        noise_vel = @SVector randn(3)
        pos_norm = norm(x0[1:3], Inf) / 100 # 1% noise
        vel_norm = norm(x0[4:6], Inf) / 1e6 # 1ppm noise

        x0 += [noise_pos * pos_norm; noise_vel * vel_norm]
        TO.set_initial_state!(prob_mpc, x0)
        
        # Log the current state
        X_traj[i+1] = x0

        # Update tracking cost
        TO.update_trajectory!(prob_mpc.obj, Z_track, k_mpc)

        # Shift the initial trajectory
        RobotDynamics.shift_fill!(prob_mpc.Z)

        # Shift the multipliers and penalties
        Altro.shift_fill!(TO.get_constraints(altro))

        # Solve the updated problem
        Altro.solve!(altro)

        # Log the results and performance
        iters[i,1] = iterations(altro)
        times[i] = altro.stats.tsolve

    end

    return X_traj, Dict(:time=>times, :iter=>iters)
end

run_MPC (generic function with 2 methods)

In [22]:
# Specify the options for the MPC solve
opts_mpc = SolverOptions(
    cost_tolerance=1e-4,
    cost_tolerance_intermediate=1e-4,
    constraint_tolerance=1e-4,
    reset_duals=false,       # often important since you don't want to re-solve for them each time
    penalty_initial=1000.0,  # a high initial penalty on these types of problems is often advantageous
    penalty_scaling=10.0,
    projected_newton=false,
    show_summary = false,    # remember to turn this off so that it doesn't print out every solve
)

# Set up the MPC problem and solve
N_mpc = 21
prob_mpc = gen_tracking_problem(prob, N_mpc);
X_traj, stats = run_MPC(prob_mpc, opts_mpc, Z_reference);

In [28]:
# Print some statistics
using Statistics # has `mean`, among many other things
@show mean(stats[:iter]);
@show mean(stats[:time]);  # ms

mean(stats[:iter]) = 5.239285714285714
mean(stats[:time]) = 0.28504210714285705
